In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np

In [2]:
vocab_size = 10000
max_length = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

x_train = pad_sequences(x_train, maxlen=max_length)
x_test = pad_sequences(x_test, maxlen=max_length)

In [3]:
model = keras.Sequential([
    layers.Embedding(vocab_size, 128, input_length=max_length),
    layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

C:\Users\suryansh\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [4]:
history = model.fit(
    x_train,
    y_train,
    epochs=3,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 1597s 5s/step - accuracy: 0.7659 - loss: 0.4845 - val_accuracy: 0.8314 - val_loss: 0.3818
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - accuracy: 0.8498 - loss: 0.3489 - val_accuracy: 0.7128 - val_loss: 0.5566
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 507s 2s/step - accuracy: 0.8735 - loss: 0.3079 - val_accuracy: 0.8610 - val_loss: 0.3639


In [5]:
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", round(accuracy, 3)) 

782/782 ━━━━━━━━━━━━━━━━━━━━ 169s 216ms/step - accuracy: 0.8580 - loss: 0.3708
Test Accuracy: 0.858


In [6]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(text_ids):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text_ids])

def predict_sentiment(review):
    tokens = [word_index.get(w, 2) for w in review.lower().split()]
    tokens_padded = pad_sequences([tokens], maxlen=max_length)
    prediction = model.predict(tokens_padded)[0][0]
    sentiment = "Positive 😀" if prediction > 0.5 else "Negative 😞"
    return sentiment

print("\nDemo Predictions:")
sample_reviews = [
    "I absolutely loved this movie, it was fantastic!",
    "The film was boring and too long.",
    "Not bad, but could have been better."
]

for r in sample_reviews:
    print(f"Review: {r}\nPredicted Sentiment: {predict_sentiment(r)}\n")


Demo Predictions:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Review: I absolutely loved this movie, it was fantastic!
Predicted Sentiment: Negative 😞

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Review: The film was boring and too long.
Predicted Sentiment: Positive 😀

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
Review: Not bad, but could have been better.
Predicted Sentiment: Negative 😞

